In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data_utility import DeepiqDataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
import torch

/vol/venv/icc24/DGL_ENV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BATCH_SIZE = 32

In [4]:
full_dataset = DeepiqDataset(
    dataset_root_dir = "./odd-one-out",
    num_panels = [5]
)

dataset_size = len(full_dataset)
print(f"Dataset size: {dataset_size}")

train_prop = 0.6
val_prop = 0.2
# test_prop = 0.2

train_size = int(train_prop * dataset_size)
val_size = int(val_prop * dataset_size)

test_size = dataset_size - train_size - val_size

print(f"Splitting into:")
print(f"  - Training set:   {train_size} samples ({train_size/dataset_size:.2%})")
print(f"  - Validation set: {val_size} samples ({val_size/dataset_size:.2%})")
print(f"  - Test set:       {test_size} samples ({test_size/dataset_size:.2%})")
print("-" * 40)


generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=generator
)


# --- 4. Verification ---
print(f"Type of train_dataset: {type(train_dataset)}")

print(f"Actual size of train_dataset: {len(train_dataset)}")
print(f"Actual size of val_dataset:   {len(val_dataset)}")
print(f"Actual size of test_dataset:  {len(test_dataset)}")

assert len(train_dataset) + len(val_dataset) + len(test_dataset) == dataset_size
print("\nSuccessfully split the dataset without losing any data points.")
print("-" * 40)


# --- 5. Create DataLoaders for each set ---
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader:   {len(val_loader)}")
print(f"Number of batches in test_loader:  {len(test_loader)}")

Dataset size: 500
Splitting into:
  - Training set:   300 samples (60.00%)
  - Validation set: 100 samples (20.00%)
  - Test set:       100 samples (20.00%)
----------------------------------------
Type of train_dataset: <class 'torch.utils.data.dataset.Subset'>
Actual size of train_dataset: 300
Actual size of val_dataset:   100
Actual size of test_dataset:  100

Successfully split the dataset without losing any data points.
----------------------------------------
Number of batches in train_loader: 10
Number of batches in val_loader:   4
Number of batches in test_loader:  4


In [5]:
print(f"Shape of a single instance in train_loader: {next(iter(train_loader))[0].shape}")

Shape of a single instance in train_loader: torch.Size([32, 5, 1, 80, 80])


In [6]:
from wren import OddOneOutWReN, train_epoch, validate, count_parameters

In [7]:
import torch
import torch.nn as nn
import time

In [8]:
IMG_SIZE = 80
LEARNING_RATE = 1e-4
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

In [9]:
import json

In [10]:
# --- Training Hyperparameters ---
N_SPLITS = 3
EPOCHS = 100
BATCH_SIZE = 32
PATIENCE = 10 # For early stopping

# --- Model & Logging Setup ---
MODEL_FOLDER_NAME = "O3_5-WReN"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# O3 STL

In [12]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutWReN(num_total_panels=5).to(DEVICE)
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        betas=(BETA_1, BETA_2),
        eps=EPSILON
    )
    criterion = nn.CrossEntropyLoss()
    scaler = None
    scheduler = None

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE, scaler=scaler)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)
    
    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Fold 1 Model has 1,212,129 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.34it/s]


  Train Loss: 1.6101 | Train Acc: 0.1767
  Val Loss:   1.6094 | Val Acc:   0.2000
  Epoch Duration: 1.92 seconds
  -> New best inner validation accuracy: 0.2000. Model state saved.

--- Fold 1, Epoch 2/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.53it/s]


  Train Loss: 1.6046 | Train Acc: 0.2580
  Val Loss:   1.6094 | Val Acc:   0.2000
  Epoch Duration: 1.30 seconds

--- Fold 1, Epoch 3/100 ---


...

  Train Loss: 0.1967 | Train Acc: 0.9364
  Val Loss:   2.8450 | Val Acc:   0.3529
  Epoch Duration: 1.33 seconds

--- Fold 3, Epoch 24/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.31it/s]


  Train Loss: 0.1511 | Train Acc: 0.9647
  Val Loss:   2.7833 | Val Acc:   0.3333
  Epoch Duration: 1.32 seconds
  Early stopping at epoch 24.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.4510 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00,  9.91it/s]


Performance of best model on the OUTER TEST SET for fold 3: 0.2831

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.3473', '0.2635', '0.2831']
Average Test Accuracy from CV: 0.2980
Standard Deviation of Test Accuracy from CV: 0.0358
-> Estimated Generalization Performance: 0.2980 ± 0.0358

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-WReN


In [13]:
def load_partial_state_dict(model_to_load: nn.Module, pretrained_path: str):
    """
    Loads weights from a pretrained model, skipping layers with mismatched shapes.

    Args:
        model_to_load (nn.Module): The model instance to load weights INTO.
        pretrained_path (str): The path to the .pt or .pth file of the pretrained model.
    """
    print(f"Loading pretrained weights from: {pretrained_path}")
    
    pretrained_dict = torch.load(pretrained_path, map_location=torch.device('cpu'))

    if 'model_state_dict' in pretrained_dict:
        pretrained_dict = pretrained_dict['model_state_dict']
    elif 'model' in pretrained_dict and isinstance(pretrained_dict['model'], dict):
        pretrained_dict = pretrained_dict['model']
        
    model_dict = model_to_load.state_dict()

    # 1. Filter out unnecessary keys and weights with mismatched shapes
    # We create a new dictionary that only contains the weights we can actually use
    filtered_dict = {}
    initialized_layers = []
    skipped_layers = []

    for key, param in pretrained_dict.items():
        if key in model_dict:
            # Check if the shape of the weights match
            if param.shape == model_dict[key].shape:
                filtered_dict[key] = param
                initialized_layers.append(key)
            else:
                # Key exists, but shapes are different. This is the main case we want to handle.
                reason = f"Shape Mismatch. Pretrained: {param.shape}, Model: {model_dict[key].shape}"
                skipped_layers.append((key, reason))
        else:
            # The key from the pretrained model doesn't exist in the new model
            skipped_layers.append((key, "Key Not Found in Model"))

    # 2. Overwrite the entries in the model's state dict with the filtered weights
    model_dict.update(filtered_dict)

    # 3. Load the updated state dict into the model.
    # Since we've filtered it, we can be strict here.
    model_to_load.load_state_dict(model_dict)

    print("\n--- Transfer Learning Report ---")
    print(f"Successfully loaded {len(initialized_layers)} / {len(model_dict)} layers.")
    if initialized_layers and len(initialized_layers) < 10:
        print("Loaded layers:")
        for layer in initialized_layers:
            print(f"  - {layer}")
    
    if skipped_layers:
        print(f"\nSkipped {len(skipped_layers)} layers:")
        for key, reason in skipped_layers:
            print(f"  - {key} ({reason})")
    print("--------------------------------\n")
    
    return model_to_load

# I-RAVEN to O3 TL

In [14]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

MODEL_FOLDER_NAME = "O3_5-WReN-TRANSFER"

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutWReN(num_total_panels=5).to(DEVICE)

    load_partial_state_dict(
        model,
        "../I_RAVEN/saved_models/best_model_wren.pth",
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        betas=(BETA_1, BETA_2),
        eps=EPSILON
    )
    criterion = nn.CrossEntropyLoss()
    scheduler = None
    scaler = None


    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE, scaler=scaler, scheduler=scheduler)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)

    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Loading pretrained weights from: ../I_RAVEN/saved_models/best_model_wren.pth

--- Transfer Learning Report ---
Successfully loaded 46 / 49 layers.

Skipped 3 layers:
  - mlp.fc3.weight (Shape Mismatch. Pretrained: torch.Size([13, 256]), Model: torch.Size([1, 256]))
  - mlp.fc3.bias (Shape Mismatch. Pretrained: torch.Size([13]), Model: torch.Size([1]))
  - proj.fc.weight (Shape Mismatch. Pretrained: torch.Size([256, 521]), Model: torch.Size([256, 517]))
--------------------------------

Fold 1 Model has 1,212,129 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.58it/s]


  Train Loss: 1.6141 | Train Acc: 0.1873
  Val Loss:   1.6094 | Val Acc:   0.2000
  Epoch Duration: 1.28 seconds
  -> New best inner validation accuracy: 0.2000. Model state saved.

--- Fold 1, Epoch 2/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.42it/s]


  Train Loss: 1.6081 | Train Acc: 0.2014
  Val Loss:   1.6092 | Val Acc:   0.2000
  Epoch Duration: 1.29 seconds

--- Fold 1, Epoch 3/100 ---


...

  Train Loss: 0.9354 | Train Acc: 0.6466
  Val Loss:   1.6453 | Val Acc:   0.2745
  Epoch Duration: 1.29 seconds

--- Fold 3, Epoch 18/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.42it/s]


  Train Loss: 0.7753 | Train Acc: 0.7420
  Val Loss:   1.7673 | Val Acc:   0.2157
  Epoch Duration: 1.29 seconds
  Early stopping at epoch 18.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.4314 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00,  9.87it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.2590

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.4611', '0.5030', '0.2590']
Average Test Accuracy from CV: 0.4077
Standard Deviation of Test Accuracy from CV: 0.1065
-> Estimated Generalization Performance: 0.4077 ± 0.1065

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-WReN-TRANSFER


# RADIO-1 to O3 TL

In [15]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

MODEL_FOLDER_NAME = "O3_5-WReN-TRANSFER_RADIO"

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")
    
    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutWReN(num_total_panels=5).to(DEVICE)

    load_partial_state_dict(
        model,
        "../RADIO/saved_models/RADIO1_WReN/deployable_model.pth",
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        betas=(BETA_1, BETA_2),
        eps=EPSILON
    )
    criterion = nn.CrossEntropyLoss()
    scheduler = None
    scaler = None


    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE, scaler=scaler, scheduler=scheduler)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)

    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Loading pretrained weights from: ../RADIO/saved_models/RADIO1_WReN/deployable_model.pth

--- Transfer Learning Report ---
Successfully loaded 46 / 49 layers.

Skipped 3 layers:
  - mlp.fc3.weight (Shape Mismatch. Pretrained: torch.Size([13, 256]), Model: torch.Size([1, 256]))
  - mlp.fc3.bias (Shape Mismatch. Pretrained: torch.Size([13]), Model: torch.Size([1]))
  - proj.fc.weight (Shape Mismatch. Pretrained: torch.Size([256, 521]), Model: torch.Size([256, 517]))
--------------------------------

Fold 1 Model has 1,212,129 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.47it/s]


  Train Loss: 1.6096 | Train Acc: 0.2085
  Val Loss:   1.6094 | Val Acc:   0.2200
  Epoch Duration: 1.27 seconds
  -> New best inner validation accuracy: 0.2200. Model state saved.

--- Fold 1, Epoch 2/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.41it/s]


  Train Loss: 1.6110 | Train Acc: 0.1802
  Val Loss:   1.6094 | Val Acc:   0.2800
  Epoch Duration: 1.28 seconds
  -> New best inner validation accuracy: 0.2800. Model state saved.

--- Fold 1, Epoch 3/100 ---


...

  Train Loss: 0.7058 | Train Acc: 0.7668
  Val Loss:   2.5892 | Val Acc:   0.1765
  Epoch Duration: 1.28 seconds

--- Fold 3, Epoch 21/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.57it/s]


  Train Loss: 0.5344 | Train Acc: 0.7986
  Val Loss:   1.8748 | Val Acc:   0.2745
  Epoch Duration: 1.30 seconds
  Early stopping at epoch 21.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.4314 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00, 10.18it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.4096

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.3054', '0.4311', '0.4096']
Average Test Accuracy from CV: 0.3821
Standard Deviation of Test Accuracy from CV: 0.0549
-> Estimated Generalization Performance: 0.3821 ± 0.0549

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-WReN-TRANSFER_RADIO


# RADIO-2 to O3 TL

In [16]:
from sklearn.model_selection import train_test_split
VAL_SPLIT_SIZE = 0.15

MODEL_FOLDER_NAME = "O3_5-WReN-TRANSFER_RADIO_2"

# ==============================================================================
# 2. DATASET & FOLD SETUP
# ==============================================================================
print(f"Total data for cross-validation: {len(full_dataset)} samples.")

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
fold_test_results = []

# ==============================================================================
# 3. CROSS-VALIDATION LOOP
# ==============================================================================
for fold, (train_outer_idx, test_outer_idx) in enumerate(kfold.split(np.zeros(len(full_dataset)))):
    print(f"\n{'='*30} OUTER FOLD {fold+1}/{N_SPLITS} {'='*30}")

    SAVED_MODELS_PATH = f"./saved_models/{MODEL_FOLDER_NAME}/"

    if not os.path.exists(SAVED_MODELS_PATH):
        os.makedirs(SAVED_MODELS_PATH)
    writer = SummaryWriter(log_dir=f'runs/{MODEL_FOLDER_NAME}/fold_{fold+1}')

    inner_train_idx, inner_val_idx = train_test_split(
        train_outer_idx,
        test_size=VAL_SPLIT_SIZE,
        shuffle=True, # Shuffle before splitting
        random_state=42
    )

    inner_train_subset = Subset(full_dataset, inner_train_idx)
    inner_val_subset = Subset(full_dataset, inner_val_idx)
    outer_test_subset = Subset(full_dataset, test_outer_idx)

    train_loader = DataLoader(inner_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(inner_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(outer_test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = OddOneOutWReN(num_total_panels=5).to(DEVICE)

    load_partial_state_dict(
        model,
        "../RADIO/saved_models/RADIO2_WReN/deployable_model.pth",
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        betas=(BETA_1, BETA_2),
        eps=EPSILON
    )
    criterion = nn.CrossEntropyLoss()
    scheduler = None
    scaler = None


    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Fold {fold+1} Model has {total_params:,} trainable parameters.")
    print(f"Fold {fold+1} Training on {len(inner_train_subset)}, Validating on {len(inner_val_subset)}, Testing on {len(outer_test_subset)}.")

    best_model_path = f"{SAVED_MODELS_PATH}/best_model_fold_{fold+1}.pth"
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\n--- Fold {fold+1}, Epoch {epoch}/{EPOCHS} ---")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE, scaler=scaler, scheduler=scheduler)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        epoch_duration = time.time() - epoch_start_time

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        print(f"  Epoch Duration: {epoch_duration:.2f} seconds")
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        writer.add_scalar('Performance/seconds_per_epoch', epoch_duration, epoch)
        writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best inner validation accuracy: {best_val_acc_fold:.4f}. Model state saved.")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    print(f"\n--- Fold {fold+1} Final Evaluation ---")
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    # Load the best model weights identified during the inner loop
    model.load_state_dict(torch.load(best_model_path))
    
    # Evaluate this best model on the completely unseen outer test set
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance of best model on the OUTER TEST SET for fold {fold+1}: {final_fold_test_acc:.4f}")

    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('Accuracy/test', final_fold_test_acc)
    
    hparams = { 'learning_rate': LEARNING_RATE, 'batch_size': BATCH_SIZE }
    final_metrics = {
        'hparam/test_accuracy': final_fold_test_acc,
        'hparam/best_inner_val_accuracy': best_val_acc_fold
    }
    writer.add_hparams(hparams, final_metrics)
    writer.close()

# ==============================================================================
# 4. AGGREGATE AND REPORT CV RESULTS
# ==============================================================================
mean_test_acc = np.mean(fold_test_results)
std_test_acc = np.std(fold_test_results)

print(f"\n{'='*30} FINAL UNBIASED PERFORMANCE SUMMARY {'='*30}")
print(f"Unbiased test accuracies from each fold: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Average Test Accuracy from CV: {mean_test_acc:.4f}")
print(f"Standard Deviation of Test Accuracy from CV: {std_test_acc:.4f}")
print(f"-> Estimated Generalization Performance: {mean_test_acc:.4f} \u00B1 {std_test_acc:.4f}")

print("\nCross-validation finished.")
print("To view TensorBoard logs, run the following command in your terminal:")
print(f"tensorboard --logdir runs/{MODEL_FOLDER_NAME}")

Total data for cross-validation: 500 samples.

============================== OUTER FOLD 1/3 ==============================
Loading pretrained weights from: ../RADIO/saved_models/RADIO2_WReN/deployable_model.pth

--- Transfer Learning Report ---
Successfully loaded 46 / 49 layers.

Skipped 3 layers:
  - mlp.fc3.weight (Shape Mismatch. Pretrained: torch.Size([13, 256]), Model: torch.Size([1, 256]))
  - mlp.fc3.bias (Shape Mismatch. Pretrained: torch.Size([13]), Model: torch.Size([1]))
  - proj.fc.weight (Shape Mismatch. Pretrained: torch.Size([256, 521]), Model: torch.Size([256, 517]))
--------------------------------

Fold 1 Model has 1,212,129 trainable parameters.
Fold 1 Training on 283, Validating on 50, Testing on 167.

--- Fold 1, Epoch 1/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.48it/s]


  Train Loss: 1.9440 | Train Acc: 0.2155
  Val Loss:   1.6095 | Val Acc:   0.2400
  Epoch Duration: 1.28 seconds
  -> New best inner validation accuracy: 0.2400. Model state saved.

--- Fold 1, Epoch 2/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.44it/s]


  Train Loss: 1.6912 | Train Acc: 0.1979
  Val Loss:   1.6095 | Val Acc:   0.1600
  Epoch Duration: 1.29 seconds

--- Fold 1, Epoch 3/100 ---


...

  Train Loss: 1.3488 | Train Acc: 0.5088
  Val Loss:   1.6093 | Val Acc:   0.3922
  Epoch Duration: 1.27 seconds

--- Fold 3, Epoch 20/100 ---


Validating: 100%|██████████| 2/2 [00:00<00:00,  4.42it/s]


  Train Loss: 1.2235 | Train Acc: 0.5442
  Val Loss:   1.5779 | Val Acc:   0.3333
  Epoch Duration: 1.31 seconds
  Early stopping at epoch 20.

--- Fold 3 Final Evaluation ---
Loading best model for fold 3 (achieved 0.4118 on inner val set).


Validating: 100%|██████████| 6/6 [00:00<00:00, 10.22it/s]

Performance of best model on the OUTER TEST SET for fold 3: 0.3735

============================== FINAL UNBIASED PERFORMANCE SUMMARY ==============================
Unbiased test accuracies from each fold: ['0.3593', '0.3353', '0.3735']
Average Test Accuracy from CV: 0.3560
Standard Deviation of Test Accuracy from CV: 0.0157
-> Estimated Generalization Performance: 0.3560 ± 0.0157

Cross-validation finished.
To view TensorBoard logs, run the following command in your terminal:
tensorboard --logdir runs/O3_5-WReN-TRANSFER_RADIO_2
